In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import plotly.express as px

In [7]:
cryptos = ["BTC-USD", "ETH-USD", "ADA-USD"]
start_date = "2020-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

data_dfs = {}
for crypto in cryptos:
    data_dfs[crypto] = yf.download(crypto, start=start_date, end=end_date)

train_ratio = 0.7
test_ratio = 0.2
split_data = {}

for crypto, df in data_dfs.items():
    total_len = len(df)
    train_end = int(total_len * train_ratio)
    test_end = train_end + int(total_len * test_ratio)
    
    split_data[crypto] = {
        "train": df.iloc[:train_end],
        "test": df.iloc[train_end:test_end],
        "later_test": df.iloc[test_end:]
    }

btc_train = split_data["BTC-USD"]["train"]
btc_test = split_data["BTC-USD"]["test"]
btc_later_test = split_data["BTC-USD"]["later_test"]

eth_train = split_data["ETH-USD"]["train"]
eth_test = split_data["ETH-USD"]["test"]
eth_later_test = split_data["ETH-USD"]["later_test"]

ada_train = split_data["ADA-USD"]["train"]
ada_test = split_data["ADA-USD"]["test"]
ada_later_test = split_data["ADA-USD"]["later_test"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
btc_train

Price,index,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
0,0,2020-01-01,7200.174316,7200.174316,7254.330566,7174.944336,7194.892090,18565664997
1,1,2020-01-02,6985.470215,6985.470215,7212.155273,6935.270020,7202.551270,20802083465
2,2,2020-01-03,7344.884277,7344.884277,7413.715332,6914.996094,6984.428711,28111481032
3,3,2020-01-04,7410.656738,7410.656738,7427.385742,7309.514160,7345.375488,18444271275
4,4,2020-01-05,7411.317383,7411.317383,7544.497070,7400.535645,7410.451660,19725074095
...,...,...,...,...,...,...,...,...
1259,1259,2023-06-13,25918.728516,25918.728516,26376.351562,25728.365234,25902.941406,14143474486
1260,1260,2023-06-14,25124.675781,25124.675781,26041.800781,24902.152344,25920.257812,14265717766
1261,1261,2023-06-15,25576.394531,25576.394531,25735.308594,24797.167969,25121.673828,15837384409


In [11]:
btc_train = btc_train.reset_index()
fig = px.line(btc_train, x="Date", y=btc_train["Close"].values.flatten(), title="BTC-USD Training Data: Close Prices Over Time")
fig.show()

In [9]:
eth_train = eth_train.reset_index()
fig = px.line(eth_train, x="Date", y=eth_train["Close"].values.flatten(), title="ETH-USD Training Data: Close Prices Over Time")
fig.show()

In [10]:
ada_train = ada_train.reset_index()
fig = px.line(ada_train, x="Date", y=ada_train["Close"].values.flatten(), title="ADA-USD Training Data: Close Prices Over Time")
fig.show()

In [ ]:
def trading_strategy1(data):
    # Calculate the 30-day Simple Moving Average (SMA)
    data['SMA_30'] = data['Close'].rolling(window=30).mean()
    # Define buy and sell signals
    data['Signal'] = None   
    # Buy signal: close > open and > SMA
    data.loc[(data['Close'] > data['Open']) & (data['Close'] > data['SMA_30']), 'Signal'] = 'Buy'
    # Sell signal: open > close and < SMA
    data.loc[(data['Open'] > data['Close']) & (data['Close'] < data['SMA_30']), 'Signal'] = 'Sell'
    # Shift signals by one day to reflect actions for the next day
    data['Action'] = data['Signal'].shift(-1)
    return data

In [ ]:
def trading_strategy2(data):
    # Calculate the 10-day Faster Moving Average and 30-day Slower Moving Average
    data['FMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_30'] = data['Close'].rolling(window=30).mean()
    # Calculate Bollinger Bands (20-day SMA and standard deviation by default)
    data['BB_Middle'] = data['Close'].rolling(window=20).mean()
    data['BB_StdDev'] = data['Close'].rolling(window=20).std()
    data['BB_Upper'] = data['BB_Middle'] + (2 * data['BB_StdDev'])
    data['BB_Lower'] = data['BB_Middle'] - (2 * data['BB_StdDev'])
    # Define buy and sell signals
    data['Signal'] = None 
    # Buy signal: FMA > SMA and Open, Close, or Low < Lower Bollinger Band
    data.loc[
        (data['FMA_10'] > data['SMA_30']) &
        ((data['Open'] < data['BB_Lower']) | (data['Close'] < data['BB_Lower']) | (data['Low'] < data['BB_Lower'])),'Signal'] = 'Buy'
    # Sell signal: SMA > FMA and Open, Close, or High > Upper Bollinger Band
    data.loc[
        (data['SMA_30'] > data['FMA_10']) &
        ((data['Open'] > data['BB_Upper']) | (data['Close'] > data['BB_Upper']) | (data['High'] > data['BB_Upper'])),'Signal'] = 'Sell'
    # Shift signals by one day to reflect actions for the next day
    data['Action'] = data['Signal'].shift(-1)
    return data

In [ ]:
def trading_strategy3(data):
    # Calculate All-Time High
    data['All_Time_High'] = data['High'].cummax()
    # Calculate Drawdown
    data['Drawdown'] = data['All_Time_High'] - data['Close']
    # Define buy and sell signals
    data['Signal'] = None  # Default value
    # Buy signal: Drawdown = 0.05 * All-Time High
    data.loc[
        data['Drawdown'] >= 0.05 * data['All_Time_High'],'Signal'] = 'Buy'
    # Sell signal: Open, Close, or High = All-Time High
    data.loc[
        (data['Open'] == data['All_Time_High']) | 
        (data['Close'] == data['All_Time_High']) | 
        (data['High'] == data['All_Time_High']),'Signal'] = 'Sell'
    # Shift signals by one day to reflect actions for the next day
    data['Action'] = data['Signal'].shift(-1)
    return data

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words='english', min_df=2)
X_train_vector = vectorizer.fit_transform(btc_train)
mlp = MLPClassifier(max_iter = 1000, random_state=8)
mlp.fit(X_train_vector, y_train)


In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Download the data (replace with the desired crypto symbol)
cryptos = ["BTC-USD"]  # Add more cryptocurrencies as needed
start_date = "2020-01-01"
end_date = "2024-01-01"

# Load data
data_dfs = {}
for crypto in cryptos:
    data_dfs[crypto] = yf.download(crypto, start=start_date, end=end_date)

# Choose one of the crypto dataframes
btc = data_dfs["BTC-USD"]

# Calculate short and long moving averages
btc['SMA10'] = btc['Close'].rolling(window=10).mean()
btc['SMA50'] = btc['Close'].rolling(window=50).mean()

# Generate buy/sell signals
btc['Signal'] = 0
btc['Signal'][10:] = [1 if btc['SMA10'][i] > btc['SMA50'][i] else -1 for i in range(10, len(btc))]
btc['Position'] = btc['Signal'].diff()

# Plot the data
plt.figure(figsize=(14, 7))
plt.plot(btc['Close'], label="BTC-USD Close Price", color='blue', alpha=0.7)
plt.plot(btc['SMA10'], label="10-day SMA", color='orange', alpha=0.7)
plt.plot(btc['SMA50'], label="50-day SMA", color='green', alpha=0.7)

# Mark Buy and Sell signals
plt.plot(btc[btc['Position'] == 1].index, 
         btc['SMA10'][btc['Position'] == 1], 
         '^', markersize=10, color='g', lw=0, label='Buy Signal')

plt.plot(btc[btc['Position'] == -1].index, 
         btc['SMA10'][btc['Position'] == -1], 
         'v', markersize=10, color='r', lw=0, label='Sell Signal')

plt.title('BTC-USD Trading Strategy with Buy/Sell Signals')
plt.legend(loc='best')
plt.show()


In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np

# Download data (replace with the desired crypto symbol)
cryptos = ["BTC-USD"]  # Add more cryptocurrencies as needed
start_date = "2020-01-01"
end_date = "2024-01-01"

# Load data
data_dfs = {}
for crypto in cryptos:
    data_dfs[crypto] = yf.download(crypto, start=start_date, end=end_date)

# Choose one of the crypto dataframes
btc = data_dfs["BTC-USD"]

# Calculate moving averages (EMA)
btc['EMA10'] = btc['Close'].ewm(span=10, adjust=False).mean()
btc['EMA50'] = btc['Close'].ewm(span=50, adjust=False).mean()

# Calculate RSI (14-period)
delta = btc['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()

rs = gain / loss
btc['RSI'] = 100 - (100 / (1 + rs))

# Generate signals
btc['Signal'] = 0
btc['Signal'] = np.where((btc['EMA10'] > btc['EMA50']) & (btc['RSI'] < 70) & (btc['RSI'] > 30), 1, 0)  # Buy signal
btc['Signal'] = np.where((btc['EMA10'] < btc['EMA50']) & (btc['RSI'] < 70) & (btc['RSI'] > 30), -1, btc['Signal'])  # Sell signal

# Backtest the strategy
btc['Position'] = btc['Signal'].shift()
btc['Daily Returns'] = btc['Close'].pct_change()
btc['Strategy Returns'] = btc['Daily Returns'] * btc['Position']

# Track trades with stop-loss and take-profit
btc['Entry Price'] = np.nan
btc['Stop Loss'] = np.nan
btc['Take Profit'] = np.nan

# Implement stop-loss and take-profit
stop_loss_pct = 0.03  # 3% stop-loss
take_profit_pct = 0.06  # 6% take-profit

# Modify the stop-loss and take-profit tracking
for i in range(1, len(btc)):
    if btc['Position'].iloc[i] == 1:  # Buy signal
        entry_price = btc['Close'].iloc[i]
        btc.loc[btc.index[i], 'Entry Price'] = entry_price
        btc.loc[btc.index[i], 'Stop Loss'] = entry_price * (1 - stop_loss_pct)
        btc.loc[btc.index[i], 'Take Profit'] = entry_price * (1 + take_profit_pct)
    elif btc['Position'].iloc[i] == -1:  # Sell signal
        btc.loc[btc.index[i], 'Entry Price'] = np.nan
        btc.loc[btc.index[i], 'Stop Loss'] = np.nan
        btc.loc[btc.index[i], 'Take Profit'] = np.nan

# Alternative method to create stop-loss and take-profit flags
btc['Stop Loss Met'] = ((btc['Stop Loss'].notna()) & (btc['Close'] <= btc['Stop Loss'])).astype(bool)
btc['Take Profit Met'] = ((btc['Take Profit'].notna()) & (btc['Close'] >= btc['Take Profit'])).astype(bool)

# Assign to DataFrame columns
btc['Stop Loss Met'] = stop_loss_met
btc['Take Profit Met'] = take_profit_met

# Plot the strategy results
plt.figure(figsize=(14, 7))
plt.plot(btc['Close'], label="BTC-USD Close Price", color='blue', alpha=0.7)
plt.plot(btc['EMA10'], label="10-day EMA", color='orange', alpha=0.7)
plt.plot(btc['EMA50'], label="50-day EMA", color='green', alpha=0.7)

# Mark Buy and Sell signals
plt.plot(btc[btc['Position'] == 1].index, 
         btc['EMA10'][btc['Position'] == 1], 
         '^', markersize=10, color='g', lw=0, label='Buy Signal')

plt.plot(btc[btc['Position'] == -1].index, 
         btc['EMA10'][btc['Position'] == -1], 
         'v', markersize=10, color='r', lw=0, label='Sell Signal')

# Mark Stop-Loss and Take-Profit
plt.plot(btc[btc['Stop Loss Met'] == True].index, 
         btc['Stop Loss'][btc['Stop Loss Met'] == True], 
         'x', markersize=10, color='black', lw=0, label='Stop Loss')

plt.plot(btc[btc['Take Profit Met'] == True].index, 
         btc['Take Profit'][btc['Take Profit Met'] == True], 
         's', markersize=10, color='purple', lw=0, label='Take Profit')

plt.title('BTC-USD Trading Strategy with Buy/Sell/Stop-Loss/Take-Profit Signals')
plt.legend(loc='best')
plt.show()

# Strategy performance (cumulative returns)
btc['Cumulative Strategy Returns'] = (1 + btc['Strategy Returns']).cumprod() - 1
plt.figure(figsize=(14, 7))
plt.plot(btc['Cumulative Strategy Returns'], label='Strategy Returns')
plt.title('Cumulative Strategy Returns')
plt.legend(loc='best')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import matplotlib.pyplot as plt

# Download crypto data (Bitcoin as an example)
crypto = "BTC-USD"
start_date = "2020-01-01"
end_date = "2024-01-01"
data = yf.download(crypto, start=start_date, end=end_date)

# Calculate moving averages
data['Short_MA'] = data['Close'].rolling(window=20).mean()  # 20-day moving average
data['Long_MA'] = data['Close'].rolling(window=200).mean()  # 200-day moving average

# Generate signals based on open and close, moving averages, and price changes
data['Signal'] = 0  # Default to no signal
data['Signal'] = np.where((data['Close'] > data['Open']) & (data['Close'] > data['Short_MA']), 1, data['Signal'])  # Buy Signal
data['Signal'] = np.where((data['Close'] < data['Open']) & (data['Close'] < data['Long_MA']), -1, data['Signal'])  # Sell Signal

# Shift signals to align with future prices (signals for the next day)
data['Position'] = data['Signal'].shift(1)

# Set up the stop-loss and take-profit rules
data['Buy_Price'] = np.nan
data['Stop_Loss'] = np.nan
data['Take_Profit'] = np.nan

for i in range(1, len(data)):
    if data['Position'].iloc[i] == 1:  # Buy signal
        data['Buy_Price'].iloc[i] = data['Close'].iloc[i]
        data['Stop_Loss'].iloc[i] = data['Close'].iloc[i] * 0.97  # 3% stop loss
        data['Take_Profit'].iloc[i] = data['Close'].iloc[i] * 1.05  # 5% take profit
    elif data['Position'].iloc[i] == -1:  # Sell signal
        data['Buy_Price'].iloc[i] = np.nan
        data['Stop_Loss'].iloc[i] = np.nan
        data['Take_Profit'].iloc[i] = np.nan

# Check if stop-loss or take-profit conditions are met
data['Stop_Loss_Met'] = data['Close'] < data['Stop_Loss']
data['Take_Profit_Met'] = data['Close'] > data['Take_Profit']

# Visualize the strategy signals, stop-loss, and take-profit on the price chart
fig = px.line(data, x=data.index, y="Close", title="Crypto Trading Strategy with Stop-Loss and Take-Profit")

# Add Buy and Sell Signals
fig.add_scatter(x=data.index[data['Position'] == 1], y=data['Close'][data['Position'] == 1],
                mode='markers', marker=dict(symbol='triangle-up', color='green', size=10), name='Buy Signal')
fig.add_scatter(x=data.index[data['Position'] == -1], y=data['Close'][data['Position'] == -1],
                mode='markers', marker=dict(symbol='triangle-down', color='red', size=10), name='Sell Signal')

# Add Stop Loss and Take Profit Lines
fig.add_scatter(x=data.index[data['Stop_Loss_Met'] == True], y=data['Stop_Loss'][data['Stop_Loss_Met'] == True],
                mode='markers', marker=dict(symbol='x', color='red', size=10), name='Stop Loss Met')
fig.add_scatter(x=data.index[data['Take_Profit_Met'] == True], y=data['Take_Profit'][data['Take_Profit_Met'] == True],
                mode='markers', marker=dict(symbol='x', color='green', size=10), name='Take Profit Met')

fig.show()

# Display the strategy results
data[['Open', 'Close', 'Short_MA', 'Long_MA', 'Position', 'Buy_Price', 'Stop_Loss', 'Take_Profit']].tail(10)
